# 어텐션의 구조

- seq2seq를 한층 더 강력하게 하는 **어텐션 매커니즘**
- 인간처럼 필요한 정보에만 "주목"할 수 있게 만든다
- seq2seq의 근본적인 문제를 해결한다

## seq2seq의 문제점

- Encoder의 출력은 "고정 길이의 벡터"
    - 고정 길이 벡터는 입력 문장의 길이에 관계없이 항상 같은 길이의 벡터로 변환하게 해준다
    - **아무리 긴 벡터여도 고정 길이에 우겨넣어짐**

<img src = "../imgs/fig 8-1.png" width="400">

## Encoder 개선

- Encoder 출력의 길이는 입력 문장에 길이에 따라 바꿔주는 것이 좋다! (당연)
- 구체적으로는, **시각별 LSTM 계층의 은닉 상태 벡터를 모두 이용하는 것**

<img src = "../imgs/fig 8-2.png" width="500">


#### <center>===> 이것으로 Encoder는 "하나의 고정 길이 벡터"라는 제약에서 벗어남</center>

- (참고) RNN 계층을 초기화할 때 두 가지 반환 중 선택 가능
    - 모든 시각의 은닉 상태 벡터 반환 
        - [keras] `return_sequences=True`
    - 마지막 은닉 상태 벡터만 반환
    
    
- 각 시각의 은닉 상태에는 직전에 입력된 단어에 대한 정보가 많이 포함되어 있다.
    - 예를 들어 `나` `는` `고양이` `로소` `이다` 에서 `고양이` 벡터에는 `나` `는` `고양이` 의 정보가 담겨 있다
    - 주변 정보를 균형있게 담기 위해서는 **양방향 RNN** 이 효과적임

## Decoder 개선 ①

<img src = "../imgs/fig 8-5.png" width="700">

- 앞 장의 Decoder는 Encoder의 LSTM 계층의 마지막 은닉 상태만을 이용한다 
- hs에서 마지막 줄만 빼내어 Decoder에 전달한 것

#### ===> hs 전부를 활용할 수 있도록 개선

---

- 사람이 문장 `나는 고양이로소이다`를 영어로 번역한다면? 
    - 나 = I, 고양이 = cat
    - 나와 고양이가 문장의 주를 이루므로 나와 고양이라는 단어에 "주목" 하면서 번역을 하게 됨. 나 = I, 고양이 = cat 와 같은 단어 간 대응 관계(`alignment`)를 미리 알고 있다면 번역 효과가 더 좋을 것임.
    - **대응 관계 (`alignment`) 아이디어 를 활용하는 것이 바로 어텐션 매커니즘**
    
---

#### 입력과 출력의 단어 간 대응 관계를 seq2seq에게 학습시키자!!!

- `도착어 단어`와 `대응 관계`에 있는 `출발어 단어`의 정보를 골라내면 더 학습(번역)이 잘 될 것 이다!
- **필요한 정보에만 주목하여 그 정보로부터 시계열 변환을 수행하는 것이 목표**



---
<img src = "../imgs/fig 8-6.png" width="700">

- `어떤 계산`이 받는 입력 두 가지
    1. Encoder로부터 받는 hs
    2. 시각 별 LSTM 계층의 은닉 상태
       - 여기서 필요한 정보만 골라 위쪽의 Affine계층으로 출력한다
- Decoder의 첫번째 계층에 마지막 은닝 상태 벡터 hs를 전달하는 것은 유지

- 목적은 **단어들의 대응관계 추출**
    - 각 시각에서 Decoder에 입력된 단어와 대응관계인 단어의 벡터를 hs에서 골라내자.
    - 예를 들어 Decoder가 `I`를 출력할 때 hs에서 `나`에 대응하는 벡터를 선택함.
        - 그러나 일부 벡터만 선택하는 것은 미분이 가능하지 않다. ( = 오차역전파 안됨 )
        - #### 일부 선택이 아니라 **모두 선택**하자. 대신 **가중치**를 별도로 계산


<img src = "../imgs/fig 8-8.png" width="600">

- 각 단어의 중요도를 나타내는 가중치 $a$
- a는 0.0 ~ 1.0 사이의 스칼라 값이며 모든 원소의 총합은 1이다
- 각 단어의 중요성을 보여주는 가중치 a와 각 단어 벡터 hs로부터 가중 합 Weighted Sum을 구하여 원하는 벡터 얻기!
    - 그 결과를 `맥락 벡터`라고 부르고 기호로는 c로 표기한다.

In [3]:
import numpy as np

T,H = 5,4
hs = np.random.randn(T,H)
a = np.array([0.8, 0.1, 0.03, 0.05, 0.02])

ar = a.reshape(5,1).repeat(4, axis=1)
print(ar.shape)

t = hs * ar 
print(t.shape)

# 단어 벡터 가중치 곱해서 합친 결과
c = np.sum(t, axis=0) 
print(c)
print(c.shape)

(5, 4)
(5, 4)
[0.6241871  1.03772847 0.41861277 0.15210266]
(4,)


In [5]:
# 미니배치 처리용 가중합
N, T, H  = 10, 5, 4
hs = np.random.randn(N, T, H)
a = np.random.randn(N, T)
ar = a.reshape(N, T, 1).repeat(H, axis=2)

t = hs * ar
print(t.shape)

c = np.sum(t, axis=1) # 1번 인덱스 축을 지워라
print(c.shape)

(10, 5, 4)
(10, 4)


In [6]:
class WeightSum:
    def __init__(self):
        self.params, self.grads = [],[]
        self.cache = None
        
    def forward(self, hs, a):
        N, T, H  = hs.shape
        
        ar = a.reshape(N, T, 1).repeat(H, axis=2)
        t = hs * ar
        c = np.sum(t, axis=1)
        
        self.cache = (hs, ar)
        return c

    def backward(self, dc):
        hs, ar = self.cache
        N, T, H = hs.shape
        
        dt = dc.reshape(N, 1, H).repeat(T, axis = 1) #sum의 역전파
        dar = dt * hs
        dhs = dt * ar
        da = np.sum(dar, axis=2) # repeat의 역전파
        
        return dhs, da

## Decoder 개선 ②

#### <center><u> 가중치 a는 어떻게 구하나? </u></center>
<img src = "../imgs/fig 8-12.png" width="400">

- h : Decoder 계층의 은닉 상태
- **h가 hs의 각 단어 벡터와 얼마나 비슷한가**를 계산
    - using 내적
    - 두 벡터 간 내적 $ a{\cdot}b $ : "두 벡터가 얼마나 같은 방향을 향하고 있는가"
    - <img src = "../imgs/fig 8-13.png" width="400">
    - hs 와 h 을 내적하여 각 단어 벡터와의 유사도 구함
    - s는 이후 소프트맥스로 0.0~1.0 사이 확률값으로 치환됨

In [8]:
import sys
sys.path.append('..')
from common.layers import Softmax
import numpy as np

N, T, H = 10,5,4
hs = np.random.randn(N, T, H)
h = np.random.randn(N, H)
hr = h.reshape(N, 1, H).repeat(T, axis=1)

t = hs * hr
print(t.shape)

s = np.sum(t, axis=2)
print(s.shape)

softmax = Softmax()
a = softmax.forward(s)
print(a.shape)

(10, 5, 4)
(10, 5)
(10, 5)


## Decoder 개선 ③

<img src= "../imgs/fig 8-16 fixed.png" width="600">

1. `Attention Weight` 계층 : Encoder의 출력인 각 단어 벡터 hs에 주목하여 가중치 a 구함
2.` Weight Sum` 계층 : a와 hs의 가중합을 구하고 맥락 벡터 c 출력

**===> `Attention` 계층**

<img src= "../imgs/fig 8-17.png" width="500">

In [9]:
class Attention:
    def __init__(self):
        self.params, self.grads = [], []
        # 1. 가중치 구하기
        self.attention_weight_layer = AttentionWeight() 
        # 2. 가중합 구하기
        self.weight_sum_layer = WeightSum()
        self.attention_weight = None

    def forward(self, hs, h):
        a = self.attention_weight_layer.forward(hs, h)
        out = self.weight_sum_layer.forward(hs, a)
        self.attention_weight = a
        return out

    def backward(self, dout):
        dhs0, da = self.weight_sum_layer.backward(dout)
        dhs1, dh = self.attention_weight_layer.backward(da)
        dhs = dhs0 + dhs1
        return dhs, dh

<img src= "../imgs/fig 8-18.png" width="700">

- **LSTM 계층과 Affine 계층 사이에 Attention 계층 삽입**

<img src= "../imgs/fig 8-19.png" width="500">

- [LSTM + Attention] 연결 벡터를 Affine 계층에 입력

---

**`Time Attention`**

<img src= "../imgs/fig 8-20.png" width="700">


In [11]:
# 시계열 방향으로 펼쳐진 다수의 Attention 계층 구현
class TimeAttention:
    def __init__(self):
        self.params, self.grads = [], []
        self.layers = None
        self.attention_weights = None

    def forward(self, hs_enc, hs_dec):
        N, T, H = hs_dec.shape
        out = np.empty_like(hs_dec)
        self.layers = []
        # 각 Attention 계층의 각 단어 가중치 보관
        self.attention_weights = []

        # Attention 계층을 필요한 수만큼 만들기
        for t in range(T): 
            layer = Attention()
            out[:, t, :] = layer.forward(hs_enc, hs_dec[:,t,:])
            self.layers.append(layer)
            self.attention_weights.append(layer.attention_weight)

        return out

    def backward(self, dout):
        N, T, H = dout.shape
        dhs_enc = 0
        dhs_dec = np.empty_like(dout)

        for t in range(T):
            layer = self.layers[t]
            dhs, dh = layer.backward(dout[:, t, :])
            dhs_enc += dhs
            dhs_dec[:,t,:] = dh

        return dhs_enc, dhs_dec

# 어텐션을 갖춘 seq2seq 구현

## Encoder 구현

# 어텐션 평가

# 8.4

# 8.5